In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### rfm là gì?
rfm là kỹ thuật phân khúc khách hàng dựa trên 3 thuộc tính: 
- (1) bao lâu rồi khách hàng chưa mua hàng (recency - R)
- (2) tuần suất mua (frequencey - F)
- (3) tổng số tiền đã mua (monetary)

#### ứng dụng rfm?
rfm hữu ích trong target marketing, thông qua việc phân loại khách hàng thành nhiều nhóm với những hành vi tương tự nhau sẽ giúp cho việc marketing trở nên hiệu quả (đúng thông điệp, đúng đối tượng)

#### bài viết này tập trung vào việc chia sẻ các bước cơ bản để làm phân tích và tạo ra dashboard để theo dõi

#### dashboard: https://datastudio.google.com/u/0/reporting/ef783a42-25e9-4d8f-b6e8-2c1d6d5b51bc/page/wHj8C

#### thực hiện phân tích rfm
- sử dụng python
- phương pháp phân khúc mỗi thuộc tính R - F - M: equal frequency binning
- gắn nhãn cho RFM: tham chiếu https://docs.clevertap.com/docs/rfm

![caption](rfm_label.png)

#### Nguồn dữ liệu: https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci
#### Các cột:
InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. 
If this code starts with the letter 'c', it indicates a cancellation.<br>
StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.<br>
Description: Product (item) name. Nominal.<br>
Quantity: The quantities of each product (item) per transaction. Numeric.<br>
InvoiceDate: Invice date and time. Numeric. The day and time when a transaction was generated.<br>
UnitPrice: Unit price. Numeric. Product price per unit in sterling (Â£).<br>
Customer ID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.<br>
Country: Country name. Nominal. The name of the country where a customer resides.<br>

#### xem qua dữ liệu và clean dựa trên các mô tả thông tin cột đã có sẵn

In [2]:
dfs = pd.read_csv(r"C:\Users\PC\Downloads\online_retail_II.csv\online_retail_II.csv")

In [3]:
dfs.head(2)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


In [4]:
dfs.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [5]:
dfs = dfs[(dfs['Customer ID'].notna()) & (dfs.Description.notna())]

In [6]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824364 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      824364 non-null  object 
 1   StockCode    824364 non-null  object 
 2   Description  824364 non-null  object 
 3   Quantity     824364 non-null  int64  
 4   InvoiceDate  824364 non-null  object 
 5   Price        824364 non-null  float64
 6   Customer ID  824364 non-null  float64
 7   Country      824364 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 56.6+ MB


#### Invoice No

In [7]:
set(dfs.Invoice.str.len())

{6, 7}

In [8]:
dfs[dfs['Invoice'].str.len() != 6].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia


In [9]:
dfs = dfs[(dfs['Invoice'].str[:1]!='C') & (dfs['Invoice'].str.isdigit())].reset_index(drop=True)

In [10]:
dfs.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


#### Stock Code

In [11]:
set(dfs.StockCode.str.len())

{1, 2, 3, 4, 5, 6, 7, 12}

In [12]:
dfs[dfs['StockCode'].str.len() != 5].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
12,489436,48173C,DOOR MAT BLACK FLOCK,10,2009-12-01 09:06:00,5.95,13078.0,United Kingdom
23,489436,35004B,SET OF 3 BLACK FLYING DUCKS,12,2009-12-01 09:06:00,4.65,13078.0,United Kingdom
28,489436,84596F,SMALL MARSHMALLOWS PINK BOWL,8,2009-12-01 09:06:00,1.25,13078.0,United Kingdom


In [13]:
dfs = dfs[(dfs['StockCode'].str.len() == 5) & (dfs['StockCode'].str.isdigit())].reset_index(drop=True)
dfs.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
2,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
3,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
4,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


#### Quantity

In [14]:
set(dfs['Quantity'].astype(str).str.isdigit())

{True}

#### datetime

In [15]:
pd.to_datetime(dfs.InvoiceDate, errors='raise')

0        2009-12-01 07:45:00
1        2009-12-01 07:45:00
2        2009-12-01 07:45:00
3        2009-12-01 07:45:00
4        2009-12-01 07:45:00
                 ...        
714043   2011-12-09 12:50:00
714044   2011-12-09 12:50:00
714045   2011-12-09 12:50:00
714046   2011-12-09 12:50:00
714047   2011-12-09 12:50:00
Name: InvoiceDate, Length: 714048, dtype: datetime64[ns]

In [16]:
dfs['InvoiceDate'] = pd.to_datetime(dfs['InvoiceDate']).dt.strftime('%Y-%m-%d')

In [17]:
# check date series
min_date = dfs.InvoiceDate.min()
max_date = dfs.InvoiceDate.max()
date_range = pd.date_range(min_date, max_date, freq='D')
date_lst = dfs.InvoiceDate.unique()
print(min_date)
print(max_date)
print(f"missing date: {[x for x in date_lst if x not in date_range]}")

2009-12-01
2011-12-09
missing date: []


#### Price

In [18]:
float == dfs['Price'].dtypes or int == dfs['Price'].dtypes

True

In [19]:
0 == any(dfs['Price'])

False

#### Customer ID

In [20]:
dfs['Customer ID'] = dfs['Customer ID'].astype(int)

In [21]:
set(dfs['Customer ID'].astype(str).str.len())

{5}

In [22]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714048 entries, 0 to 714047
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      714048 non-null  object 
 1   StockCode    714048 non-null  object 
 2   Description  714048 non-null  object 
 3   Quantity     714048 non-null  int64  
 4   InvoiceDate  714048 non-null  object 
 5   Price        714048 non-null  float64
 6   Customer ID  714048 non-null  int32  
 7   Country      714048 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 40.9+ MB


In [23]:
dfs['Revenue'] = dfs['Price'] * dfs['Quantity']

In [24]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714048 entries, 0 to 714047
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      714048 non-null  object 
 1   StockCode    714048 non-null  object 
 2   Description  714048 non-null  object 
 3   Quantity     714048 non-null  int64  
 4   InvoiceDate  714048 non-null  object 
 5   Price        714048 non-null  float64
 6   Customer ID  714048 non-null  int32  
 7   Country      714048 non-null  object 
 8   Revenue      714048 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 46.3+ MB


#### ok. đã đảm bảo đủ thông tin khách hàng với các thuộc tính đúng

In [25]:
dfs['Customer ID'].nunique()

5818

#### tiến hành tính toán

In [26]:
def create_rfm_dataframe(dfs):
    last_date_in_dfs = dfs.InvoiceDate.max()
    ddf = dfs.groupby('Customer ID').agg(
        Recency=('InvoiceDate', lambda x: (pd.to_datetime(last_date_in_dfs) - pd.to_datetime(x.max())).days),
        Frequency=('InvoiceDate', lambda x: x.nunique()),
        Monetary=("Revenue", np.sum)
    ).reset_index()
    return ddf
rfm_dfs = create_rfm_dataframe(dfs)
rfm_dfs.head()

,Customer ID,Recency,Frequency,Monetary
0,12346,325,3,77327.62
1,12347,2,8,4620.32
2,12348,75,5,1658.40
3,12349,18,3,3319.06
4,12350,310,1,258.00


In [27]:
def calc_rfm_score(dfs):
    """
    method: equal frequency binning -> pandas function: pandas.qcut()
    lowest 1 -> 5 highest
    Recency = 5: Mới mua hàng gần đây
    Frequency = 5: Tần suất mua cao
    Monatary = 5: Chi tiêu nhiều
    """
    dfs['r_score'] = pd.qcut(dfs['Recency'].rank(method='first'), q=5, labels=[5, 4, 3, 2, 1])
    dfs['f_score'] = pd.qcut(dfs['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
    dfs['m_score'] = pd.qcut(dfs['Monetary'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
    dfs['rfm_score'] = dfs['r_score'].astype(str)+dfs['f_score'].astype(str)+dfs['m_score'].astype(str)
    return dfs

In [28]:
ddf = calc_rfm_score(rfm_dfs)
ddf.head()

,Customer ID,Recency,Frequency,Monetary,r_score,f_score,m_score,rfm_score
0,12346,325,3,77327.62,2,3,5,235
1,12347,2,8,4620.32,5,5,5,555
2,12348,75,5,1658.40,3,4,4,344
3,12349,18,3,3319.06,5,3,5,535
4,12350,310,1,258.00,2,1,2,212


### gắn nhãn

![caption](user_segments.png)

In [29]:
# việc lựa chọn các rfm để áp vào các phân loại là dựa trên các thuộc tính  R - F - M

In [49]:
champions = ['555', '554', '545', '544', '455', '454', '445', '444']
loyal_customers = ['553', '543', '453', '443', '355', '354', '345', '344']
potential_loyalist = [
    '552', '542', '541', '535', '534', '533', '532', '531', '452', '442', 
    '441', '435', '434', '433', '432', '431', '353', '352', '351', '451', 
    '423', '413', '551', '525', '524', '515', '512']
about_to_sleep = ['331', '322', '321', '312', '311', '242', '241', '232', '231', '221']
at_risk = ['255', '254', '253', '252', '251', '245', '244', '243', '235', '234', '233', '225', '215']
cannot_lose_them = ['155', '154', '145', '144', '135', '125', '124', '114']
hibernating = [
    '223', '222', '212', '211', '153', '152', '142', '134', 
    '133', '132', '123', '131', '122', '121', '113', '112', 
    '111', '224', '214', '213', '151', '143', '141']
need_attention = ['343', '342', '341', '335', '334', '333', '332', '325', '324', '323', '315', '314', '313']
new_customers = ['522', '521', '511', '422', '421', '412', '411']
promissing = ['523', '514', '513', '425', '424', '415', '414', '115']

In [50]:
cond = [
    ddf['rfm_score'].isin(about_to_sleep),
    ddf['rfm_score'].isin(at_risk),
    ddf['rfm_score'].isin(cannot_lose_them),
    ddf['rfm_score'].isin(champions),
    ddf['rfm_score'].isin(hibernating),
    ddf['rfm_score'].isin(loyal_customers),
    ddf['rfm_score'].isin(need_attention),
    ddf['rfm_score'].isin(new_customers),
    ddf['rfm_score'].isin(potential_loyalist),
    ddf['rfm_score'].isin(promissing)
    
]

choice = [
    'About to Sleep',
    'At Risk',
    'Cannot Lose Them',
    'Champions',
    'Hibernating',
    'Loyal Customers',
    'Need Attention',
    'New Customers',
    'Potential Loyalists',
    'Promising',
]

ddf['segmt'] = np.select(cond, choice, 'undefined')

In [51]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5818 entries, 0 to 5817
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Customer ID  5818 non-null   int64   
 1   Recency      5818 non-null   int64   
 2   Frequency    5818 non-null   int64   
 3   Monetary     5818 non-null   float64 
 4   r_score      5818 non-null   category
 5   f_score      5818 non-null   category
 6   m_score      5818 non-null   category
 7   rfm_score    5818 non-null   object  
 8   segmt        5818 non-null   object  
dtypes: category(3), float64(1), int64(3), object(2)
memory usage: 290.5+ KB


In [52]:
ddf[ddf['segmt']=='undefined'].rfm_score.unique()

array([], dtype=object)

In [53]:
ddf.groupby('segmt')['Customer ID'].nunique()

segmt
About to Sleep          538
At Risk                 420
Cannot Lose Them         86
Champions              1271
Hibernating            1563
Loyal Customers         573
Need Attention          459
New Customers           286
Potential Loyalists     562
Promising                60
Name: Customer ID, dtype: int64

In [54]:
(ddf.groupby('segmt')['Customer ID'].nunique()/ddf['Customer ID'].nunique()).sort_values(ascending=False)

segmt
Hibernating            0.268649
Champions              0.218460
Loyal Customers        0.098487
Potential Loyalists    0.096597
About to Sleep         0.092472
Need Attention         0.078893
At Risk                0.072190
New Customers          0.049158
Cannot Lose Them       0.014782
Promising              0.010313
Name: Customer ID, dtype: float64

In [55]:
5818==ddf['Customer ID'].nunique()

True

In [56]:
cols = ['Customer ID', 'Recency','Frequency', 'Monetary', 'r_score', 'f_score','m_score', 'rfm_score', 'segmt']

In [59]:
ddf[cols].to_excel("rfm_data1.xlsx", index=False)

In [60]:
for v in ddf.segmt.unique():
    print(v)
    print(ddf[ddf['segmt']==v].rfm_score.unique())
    print(ddf[ddf['segmt']==v].head().to_markdown())
    print("\n")
    print("-"*100)

At Risk
['235' '233' '243' '245' '234' '215' '255' '225' '244' '254' '253']
|    |   Customer ID |   Recency |   Frequency |   Monetary |   r_score |   f_score |   m_score |   rfm_score | segmt   |
|---:|--------------:|----------:|------------:|-----------:|----------:|----------:|----------:|------------:|:--------|
|  0 |         12346 |       325 |           3 |   77327.6  |         2 |         3 |         5 |         235 | At Risk |
| 27 |         12373 |       311 |           3 |     869.15 |         2 |         3 |         3 |         233 | At Risk |
| 31 |         12377 |       315 |           4 |    2991.78 |         2 |         3 |         5 |         235 | At Risk |
| 42 |         12389 |       402 |           3 |    1070.33 |         2 |         3 |         3 |         233 | At Risk |
| 55 |         12402 |       323 |           6 |    1039.4  |         2 |         4 |         3 |         243 | At Risk |


--------------------------------------------------------------------